# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from sql_queries import *
from refresh_database import drop_tables

#### Refresh all the tables in the database

In [2]:
!python refresh_database.py

Refresh completed!!!


#### Creating list of filepaths to process original event csv data files

In [3]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [5]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [6]:
# Making connecto to our cassandra db

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [7]:
#Create keyspace sparkifydb if it does not exist

session.execute("""
        CREATE KEYSPACE IF NOT EXISTS sparkifydb
        WITH REPLICATION =
        { 'class' : 'SimpleStrategy', 'replication_factor': 1}"""
        )


#### Set Keyspace

In [8]:
#Set KEYSPACE to the keyspace to our sparkifydb
session.set_keyspace('sparkifydb')


### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




## Query #1 : 
### Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

By looking at query #1, we know our customer sparkify is trying to find out individual song's information giving a sessionId and itemInSession value.  For this reason, we are going to use sessionId and itemInSession as our primary key.  

Once we have our primary key determined, we need to only include the asking columns into our table.  

So our table for query #1 looks like the following

- sessionId int
- itemInSession int
- artist text
- song text
- song_length decimal

In [9]:
#sample query giving 338 as sessionId and 4 as itemInSession
print(select_by_sessionId_itemInSession % (338, 4))

                    


    SELECT artist, song, song_length FROM music_history_by_sessionId_itemInSession WHERE sessionId=338 AND itemInSession=4



In [10]:
# open up the file and parse the data and insert into table line by line
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = insert_music_history_sessionId_itemInSession
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [11]:
#We are using select statement validate the data we inserted into the table
query = select_by_sessionId_itemInSession % (589, 10)
print(query)
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.song_length)
del rows


    SELECT artist, song, song_length FROM music_history_by_sessionId_itemInSession WHERE sessionId=589 AND itemInSession=10

In Flames Like you better dead 202.84036


## Query #2 : 
### Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

By looking at query #2, we know our customer sparkify is trying to find out our user's perference of songs giving a session.  Therefore, our primary key would be a composite key including userId and sessionId.  And since we need to sort by itemInSession, we will add that as our cluster key.  

Once we have our primary key and clustering column determined, we need to only include the asking columns into our table.  

So our table for query #2 looks like the following

- userId int
- sessionId int
- artist text
- song text
- firstname text
- lastname text

In [12]:
#sample query giving 10 as userId and 182 as sessionId
print(select_by_userId_sessionId % (10, 182))

                    


    SELECT artist, song, firstName, lastName FROM music_history_by_userId_session_d WHERE userId=10 AND sessionId=182



In [13]:
# open up the file and parse the data and insert into table line by line
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:        
        query = insert_music_history_userId_sessionId
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [14]:
##We are using select statement validate the data we inserted into the table

query = select_by_userId_sessionId % (29, 589)
print(query)
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    #print(row)
    print (row.artist, row.song, row.firstname, row.lastname)
del rows


    SELECT artist, song, firstName, lastName FROM music_history_by_userId_session_d WHERE userId=29 AND sessionId=589

Lil Jon / The East Side Boyz / DJ Flexx Aww Skeet Skeet Jacqueline Lynch
Enya The Sun In The Stream Jacqueline Lynch
Furthermore We Need To Talk (She And I Album Version) Jacqueline Lynch
Kid Cudi / MGMT / Ratatat Pursuit Of Happiness (nightmare) Jacqueline Lynch
Crosby_ Stills & Nash Haven't We Lost Enough?  (LP Version) Jacqueline Lynch
Passion Pit Sleepyhead Jacqueline Lynch
Keith Sweat Merry Go Round (Remastered Single Version) Jacqueline Lynch
In Flames Like you better dead Jacqueline Lynch
The All-American Rejects Too Far Gone Jacqueline Lynch
Sick Puppies Don't Walk Away Jacqueline Lynch
Rapsusklei Nube Inerte (con Aniki) Jacqueline Lynch
Jonezetta Untitled Jacqueline Lynch
Cut Copy A Dream Jacqueline Lynch
BeyoncÃÂ© Halo Jacqueline Lynch
Easy Star All-Stars Getting Better (feat. The Mighty Diamonds) Jacqueline Lynch
Blue Man Group Sing Along [Featuring Dave M

## Query #3 : 
### Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

By looking at query #3, we know our customer sparkify is trying to find out all the users (with their first and last name) that has listened to a given song.  We know song is a must in the partition key.  But since song itself cannot uniquely identify a record, we need to include userId into the parition key.  Therefore, our primary key would be a composite key including song and userId.  

Once we have our primary key determined, we need to only include the asking columns into our table.  

So our table for query #3 looks like the following

- song text
- userId int
- firstname text
- lastname text

In [15]:
#sample query giving 'All Hands Against His Own' as song title
print(select_by_song % ('All Hands Against His Own'))


SELECT firstName, lastName FROM music_history_by_song where song='All Hands Against His Own'



In [16]:
# open up the file and parse the data and insert into table line by line
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = insert_music_history_song
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [17]:
#We are using select statement validate the data we inserted into the table

query = select_by_song % "Like you better dead"
print(query)
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    #print(row)
    print (row.firstname, row.lastname)
del rows


SELECT firstName, lastName FROM music_history_by_song where song='Like you better dead'

Jacqueline Lynch


### Drop the tables before closing out the sessions

In [18]:
##Drop the table to clear the development environment
drop_tables(session)

### Close the session and cluster connection¶

In [19]:
session.shutdown()
cluster.shutdown()